# **Título: Lab2**

**Membros da Equipe do Projeto:**
    
    - Pedro Ulisses 
    
    - Iago Jacob

    - Welberson Franklin

Primeiramente, importando os módulos que serão utilizados no pré-processamento:

In [ ]:
## Este módulo é para trabalhar os datasets, neste caso de imagens, com a unidade fundamental dos tensores
import torch

## Esse submódulo de torch é para criar o dataset a partir de uma pasta no computador e depois carregá-lo
from torch.utils.data import Dataset, DataLoader

## Estes módulo é para realizar transformações sobre as imagens
import torchvision.transforms as transforms
from PIL import Image

## Estes módulos são as redes neurais, já treinadas, que utilizaremos para gerar características para as imagens do dataset
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.models import resnet50, ResNet50_Weights

Depois, importando-se os utilizados para o restante das atividades:

In [ ]:
## Este módulo permite a leitura de arquivos no python e operações com caminhos
import os

## Estes módulos serão utilizados para selecionar os melhores atributos para o posterior treino da AM
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

## Estes módulos serão utilizados para criar as figuras com os boxplots
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Estes módulos serão utilizados para fazer o treinamento dos modelos de AM
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


        As redes neurais utilizadas para o processamento foram ResNet18 e ResNet50. O motivo pelos quais elas foram escolhidas é para isolar o efeito que a adição de camadas a uma rede neural tem sobre os resultados de acurácia nos testes. Tal efeito pode ser isolado porque as ambas as redes seguem a arquitetura ResNet. 

Abaixo, temos o trecho de código que faz a extração de atributos das imagens de cada conjunto de dados. Para especificar a rede neural, escrevemos:

-  rede_neural = 'rn18' ou 'rn50'

Para especificar o conjunto de dados que está sendo pré-processado, escrevemos:

- conj_pre_process = 'train' ou "validation" ou "test"

In [ ]:
rede_neural = 'rn18'
conj_pre_process = 'train'


## Este objeto transform é uma pipeline de ações para padronizar o tipo de imagem do nosso dataset, retornando tensores do torch ao final
transform = transforms.Compose([
    transforms.Resize(256),         
    transforms.CenterCrop(224),  
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])


## Classe criada para realizar criar o dataset a partir dos caminhos fornecidos na máquina onde este código é rodado
class TxtFileDataset(Dataset):
    def __init__(self, txt_file, root_dir, transform=None):

        self.root_dir = root_dir
        self.transform = transform
        self.image_paths_and_labels = []

        with open(txt_file, 'r') as f:
            for line in f:
                # Divide cada linha em uma lista com o caminho e com o rótulo, respectivamente
                line = line.strip()
                if line:
                    path, label = line.split()
                    self.image_paths_and_labels.append((path, int(label)))

    def __len__(self):
        # Retorna o número total de amostras no dataset
        return len(self.image_paths_and_labels)

    def __getitem__(self, idx):
        ## Será útil para situações em que precisaremos fazer dataset[idx]

        relative_path, label = self.image_paths_and_labels[idx]
        
        full_image_path = os.path.join(self.root_dir, relative_path)
        
        # Carrega a imagem usando a biblioteca Pillow (PIL)
        # .convert('RGB') garante que a imagem tenha 3 canais
        image = Image.open(full_image_path).convert('RGB')

        # Aplica as transformações na imagem, se houver
        if self.transform:
            image = self.transform(image)

        return image, label


################################################################### 

caminho_arquivo_txt = f'.{os.sep}Dados{os.sep}{conj_pre_process}.txt'        # caminho para o conjunto de dados a ser pré-processado
diretorio_raiz_dados = f'.{os.sep}Dados'                                     # caminho para o diretório de Dados

################################################################### 

## Criação do dataset
dataset_train = TxtFileDataset(txt_file=caminho_arquivo_txt,
                             root_dir=diretorio_raiz_dados,
                             transform=transform)

## Estabelece-se que será utilizada a CPU para os processos com as redes neurais
device = torch.device("cpu")

## Para obtermos um processamento mais rápido, rodaremos as redes neurais em lotes de 16 imagens
batch_size = 16

data_loader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)

################################################################### 

## Aqui, escolhemos a rede neural que será utilizada

if rede_neural == 'rn18':
    resnet = resnet18(weights=ResNet18_Weights.DEFAULT).to(device)
elif rede_neural == 'rn50':
    resnet = resnet50(weights=ResNet18_Weights.DEFAULT).to(device)
else:
    raise(NameError('Escolha uma rede neural dentre as duas citadas no texto.'))

################################################################### 


## Estabelecemos que não iremos treinar a rede, mas testá-la, avaliá-la, tomando o seu tensor intermediário logo antes da classificação final
resnet.eval()
extracted_features = []

## Definimos a função hook para extrair o tensor intermediário do processo das redes neurais

def hook_fn(module, input, output):
    extracted_features.append(output.detach().cpu())


# Registrando o hook na camada 'avgpool'(camada antes da camada de classificação)
hook_handle = resnet.avgpool.register_forward_hook(hook_fn)
all_labels = []


## Aplicação da rede neural sobre as imagens no dataset já carregado
with torch.no_grad():
    for images, labels in data_loader:
        images = images.to(device)
        _ = resnet(images)
        all_labels.append(labels)

## Ajuste da dimensão dos tensores que guardam o resultado intermediário das redes neurais quando alimentadas com o dataset
features = torch.cat(extracted_features, dim=0)
features = features.view(features.size(0), -1)
labels = torch.cat(all_labels)

## Salvando os resultados obtidos para acessá-los diretamente depois
torch.save({
    'features': features,
    'labels': labels
}, rede_neural+'_'+conj_pre_process+'.pt')


print("Features extraídas:", features.shape) 
print("Labels:", labels.shape) 
hook_handle.remove()

No trecho de código abaixo, fazemos a seleção dos cinco atributos mais representativos para a rotulação formiga/abelha. Escolhemos a rede neural que produz atributos escrevendo:

-  rede_neural = 'rn18' ou 'rn50'

Especificamos o conjunto de dados no qual faremos a seleção de cinco atributos escrevendo:

- conj_dados_kselect = 'train' ou "validation" ou "test"

In [ ]:
rede_neural = 'rn18'
conj_dados_kselect = 'train'

# Carrega o dicionário salvo previamente salvo em arquivo
dados_carregados = torch.load(rede_neural+'_'+conj_dados_kselect+'.pt')

# Acessa os tensores usando as mesmas chaves
features = dados_carregados['features']
labels = dados_carregados['labels']

# Feature extraction
test = SelectKBest(score_func=chi2,k=5)
fit = test.fit(features, labels)

## Com esta função, obtém-se um vetor que mostra quais foram as features selecionadas
mask = fit.get_support() 
print(len(mask))

features = fit.transform(features)
labels = labels.numpy()


## Salvando os resultados obtidos, já no formato de array numpy, para acessá-los diretamente depois
from scipy.io import savemat
savemat('selected_'+rede_neural+'_'+conj_dados_kselect+'.mat', {'features': features, 'labels':labels, 'mask': mask  })


print("Features extraídas:", features.shape) 
print("Labels:", labels.shape)


Definindo-se uma função que gera um box plot para cada um dos cinco atributos, dado o nome de um conjunto de dados cujos atributos foram selecionados:

In [ ]:
from scipy.io import loadmat


def criar_box_plot(idx, nome_arquivo):

    # Carrega o dicionário salvo previamente salvo em arquivo
    dados_carregados = loadmat(nome_arquivo)

    # Carrega os arrays numpy
    features = dados_carregados['features']
    labels = dados_carregados['labels'].squeeze()

    feature_a_plotar = features[:, idx]

    # Dataframe do pandas com duas colunas: uma para os valores da feature e outra para os rótulos.
    df = pd.DataFrame({
        'feature_value': feature_a_plotar,
        'label': labels
    })

    plt.figure(figsize=(10, 6))

    sns.boxplot(x='label', y='feature_value', data=df)

    plt.title(f'Boxplot da Feature {idx} para cada Classe')
    plt.xlabel('Classe (Label)')
    plt.ylabel(f'Valor da Feature {idx}')
    plt.grid(True) # Adiciona uma grade para facilitar a leitura
    plt.show()

Primeiramente, para a rede neural ResNet18, obtém-se os seguintes box plots para os cinco atributos selecionados:

In [ ]:
criar_box_plot(0, 'selected_rn18_train.mat')

In [ ]:
criar_box_plot(1, 'selected_rn18_train.mat')

In [ ]:
criar_box_plot(2, 'selected_rn18_train.mat')

In [ ]:
criar_box_plot(3, 'selected_rn18_train.mat')

In [ ]:
criar_box_plot(4, 'selected_rn18_train.mat')

Desta vez, para a rede neural ResNet50:

In [ ]:
criar_box_plot(0, 'selected_rn50_train.mat')

In [ ]:
criar_box_plot(1, 'selected_rn50_train.mat')

In [ ]:
criar_box_plot(2, 'selected_rn50_train.mat')

In [ ]:
criar_box_plot(3, 'selected_rn50_train.mat')

In [ ]:
criar_box_plot(4, 'selected_rn50_train.mat')

        Idealmente, as classes seriam facilmente distinguíveis por um atributo se seus box plots, incluindo os "whiskers", não se intersectassem. Isso demonstraria que a distribuição de valores desse atributo para cada uma das classes não tem muitas coincidências para as classes, de modo que elas possam ser distinguíveis. No caso dos box plots mostrados acima, nota-se que aqueles produzidos pelos atributos da ResNet18 têm consideráveis intersecções, sendo a largura interquartis de ambas classes relativamente grandes, de modo a gerar coincidências. Para os box plots do ResNet50, nota-se que em quase todas os atributos há uma classe com largura interquartis relativamente pequena, com menos intersecções entre os box plots de cada classe, de modo que se possa esperar que as previsões do ResNet50 tenham mais acurácia que as do ResNet18.

        Agora, iremos realizar o treinamento de um modelo com diferentes algoritmos, utilizando-se um conjunto de treinamento, outro de validação para o ajuste de hiperparâmetros e outro de teste, para enfim avaliar a acurácia do modelo obtido. Com esses dados de acurácia, será possível não só comparar os algoritmos, mas a forma como a escolha da rede neural do pré-processamento lhes afeta a acurácia.

Abaixo, começamos com o algoritmo KNN:

In [ ]:


rede_neural = 'rn18'


nome_treino    = rede_neural+'_train.pt'
nome_validacao = rede_neural+'_validation.pt'
nome_teste = rede_neural+'_test.pt'

selecionar_hiper_param = {

    1: [ None, 0] , 
    3: [ None, 0] ,
    5: [ None, 0] ,
#   valor_k:  Pipeline, acurácia
}

i_max_acuracia = 1

for i in [1, 3, 5]:

    pipeline_knn = Pipeline([
        ('scaler', MinMaxScaler()),
        ('knn', KNeighborsClassifier(n_neighbors=i)) # Exemplo de parâmetros
    ])

    dados_treino = torch.load(nome_treino)
    X_treino = dados_treino['features']
    y_treino = dados_treino['labels']

    dados_validacao = torch.load(nome_validacao)
    X_validacao = dados_validacao['features']
    y_validacao = dados_validacao['labels']

    ## Faz o treinamento
    pipeline_knn.fit(X_treino, y_treino)

    ## Calcula a acurácia para o conjunto de validação
    previsoes = pipeline_knn.predict(X_validacao)
    acuracia = accuracy_score(y_validacao, previsoes)

    selecionar_hiper_param[i] = [pipeline_knn, acuracia]

    if selecionar_hiper_param[i_max_acuracia][1] < acuracia:
        i_max_acuracia = i 



melhor_knn = selecionar_hiper_param[i_max_acuracia][0]
dados_teste = torch.load(nome_teste)
X_teste = dados_treino['features']
y_teste = dados_treino['labels']
previsoes_teste = pipeline_knn.predict(X_teste)
acuracia_teste = accuracy_score(y_teste, previsoes_teste)


print( f'Para o treino com o conjunto de dados {nome_treino} e validação {nome_validacao}, obteve-se maior acurácia, de {selecionar_hiper_param[i_max_acuracia][1]:.4f} com k={i_max_acuracia}.')
print( f'\n\nPara o treino com o conjunto de dados {nome_treino} com k={i_max_acuracia}, a acurácia do modelo no conjunto teste foi de {acuracia_teste:.4f}.')

Agora, para o algoritmo AD:

Por último, para o algoritmo RF:

Enfim, fazendo-se o treinamento e o teste de cada algoritmo, consguimos preencher a tabela de acurácia, a seguir:

| | kNN | AD | RF |
| :--- | :--- | :--- | :--- |
| **Atributos ResNet18** | 0.9118 | <valor acurácia> | <valor acurácia> |
| **Atributos ResNet50** | 0.9353 | <valor acurácia> | <valor acurácia> |

2. Treinamento dos modelos
a) kNN
b) AD
c) RF
3. Teste dos modelos
4. Discussões
5. Conclusões: Comentários e sugestões sobre o trabalho
(complexidade/facilidade, sugestões, etc.).



